<img src='pics/otus.png'>

In [5]:
from __future__ import print_function
from collections import defaultdict
import pandas as pd
import numpy as np
import matplotlib
import pandas_profiling
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns

%matplotlib inline
plt.rcParams["figure.figsize"] = (15, 8)
pd.options.display.float_format = '{:.2f}'.format

In [8]:
fn = 'orders.csv'

In [9]:
df = pd.read_csv(fn)
df.head(5)

,DISCOUNT_VALUE,NAME,PRICE,QUANTITY,date_insert,discount,itemID,orderID,ordered_item_count,price,user_id
0,23.55%,Хроники Амбера. Том 2,357.00,1,2017-03-19 22:25:20,110.00,169108,602051,1,527.00,911531
1,4.07%,Хроники Амбера. Том 1,448.00,1,2017-03-19 22:19:58,19.00,1394744,602047,1,618.00,911531
2,4.17%,Пищеводитель,414.00,1,2017-03-19 20:58:00,65.00,191276,602017,4,1503.00,911587
3,4.17%,Аудиокн. Мясников. &quot;Ржавчина&quot;: что д...,253.00,1,2017-03-19 20:58:00,65.00,157945,602017,4,1503.00,911587
4,4.09%,Инфекции. Как защитить себя и своего ребенка,422.00,1,2017-03-19 20:58:00,65.00,205173,602017,4,1503.00,911587


# 1. Провести EDA
Просмотреть структуру данных, значения каждого признака, поискать особенности в признаках и в данных вцелом

In [10]:
"""
формула рассчета изначальной цены за книгу (без скидки)
"""
def book_price(price, discount):
    #price = nominal_price - (nominal_price/100) * discount
    #price = nominal_price * (1 - discount/100)
    nominal_price = price/(1 - discount/100)
    return nominal_price
    

In [11]:
print(book_price(263,20.06))
print(book_price(316,3.95))

328.9967475606705
328.99531494013536


In [12]:
print(df.count())
df.describe()
df.sort_values(['NAME', 'date_insert']).head(5)

DISCOUNT_VALUE        553777
NAME                  557811
PRICE                 557818
QUANTITY              557818
date_insert           557818
discount              557818
itemID                557818
orderID               557818
ordered_item_count    557818
price                 557818
user_id               557818
dtype: int64


,DISCOUNT_VALUE,NAME,PRICE,QUANTITY,date_insert,discount,itemID,orderID,ordered_item_count,price,user_id
60033,20.06%,"# Мужчины с Марса, женщины с Венеры. Курс испо...",263.00,1,2017-03-31 15:54:33,133.00,1418863,610585,4,1138.00,917439
63068,3.95%,"# Мужчины с Марса, женщины с Венеры. Курс испо...",316.00,1,2017-04-04 17:32:24,256.00,1418863,612159,5,1285.00,220005
128639,23.1%,"# Мужчины с Марса, женщины с Венеры. Курс испо...",253.00,1,2017-05-02 23:22:57,2066.00,1418863,626643,32,7144.00,38770
130829,23.1%,"# Мужчины с Марса, женщины с Венеры. Курс испо...",253.00,2,2017-05-02 23:27:03,604.00,1418863,626649,9,5349.00,38770
135324,10.03%,"# Мужчины с Марса, женщины с Венеры. Курс испо...",329.00,1,2017-05-11 18:44:04,187.00,1418863,630919,4,1058.00,931194


In [17]:
df[df['NAME'].isnull()]

,DISCOUNT_VALUE,NAME,PRICE,QUANTITY,date_insert,discount,itemID,orderID,ordered_item_count,price,user_id
110107,NaN,NaN,11221.00,1,2017-04-27 09:15:19,0.00,132542,624656,1,11500.00,195867
146016,NaN,NaN,12587.00,1,2017-05-19 02:15:11,0.00,133342,634448,1,12866.00,195867
159453,NaN,NaN,12587.00,1,2017-05-23 11:15:12,0.00,133342,636646,1,12587.00,195867
160684,NaN,NaN,1625.00,1,2017-05-26 12:15:14,0.00,144934,639120,1,1625.00,195867
178111,NaN,NaN,1625.00,1,2017-05-29 09:15:12,0.00,144934,640360,1,1625.00,195867
200009,NaN,NaN,1625.00,1,2017-06-06 12:15:11,0.00,144934,645680,1,1625.00,195867
200772,NaN,NaN,1625.00,1,2017-06-09 06:15:14,0.00,144934,647375,1,1625.00,195867


1. нужно NaN значения DISCOUNT_VALUE привести к 0
2. NaN  значения NAME удалить 
3. DISCOUNT_VALUE  привести к цифровому значению
4. Добавить колонку book_price  где будет цена книги без скидки

# 2. Спланировать эксперимент 

Задача:
* сегментация товаров + сегментация пользователей (по товарам, по поведению, скидкам и пр.)

Дополнительнительные задачи (опционально, подумайте о них, могут появиться интересные идеи и находки):
* поиск товаров-заменителей и товаров, дополняющих друг друга
* поиск выбросов среди товаров
* поиск выбросов среди заказов

Эксперимент - составить план, по которому будет проделана исследовательская работа. Примерные пункты плана:
* Составить первичный список признаков, необходимых для решения задачи
* Сгенерировать список новых признаков, которые можно получить из данных
* Применить понижение размерности для визуализации данных
* Применить кластеризацию, визуализировать
* Описание кластера
* Любые другие пункты

# 3. Выполнить подготовку данных для построения модели

Выполнить необходимые преобразования данных

In [25]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline

class TransformPercent(BaseEstimator, TransformerMixin):
    
    def __init__(self, *args):
        self._field_name = args[0]
    
    def fit(self, df, y=None):
        return self
    
    def transform(self, df,  y=None):
        _df = df.copy()  # делаем копию данных (полагая, что df - датафрейм pandas),   
        _df[self._field_name] = _df[self._field_name].replace('[\%]','',regex=True).astype(float)
        _df[self._field_name].fillna(0.0)
        return _df    
    

In [27]:
model_pipe = Pipeline([('percent', TransformPercent('DISCOUNT_VALUE'))])
model_pipe.fit(df)
analyze_df = model_pipe.transform(df)

,DISCOUNT_VALUE,NAME,PRICE,QUANTITY,date_insert,discount,itemID,orderID,ordered_item_count,price,user_id
11,0.00,Феномен Инстаграма. Как раскрутить свой аккаун...,254.15,1,2017-03-19 20:22:19,44.85,1423007,602002,1,384.15,911573
17,0.00,Феномен Инстаграма. Как раскрутить свой аккаун...,254.15,10,2017-03-19 19:16:27,44.85,1423007,601981,1,2541.50,911556
29,0.00,Приключения барона Мюнхгаузена (ил. Л.Каплана),456.50,1,2017-03-19 17:18:03,584.63,669407,601928,5,2854.37,483101
30,0.00,Пеппи Длинныйчулок,345.28,1,2017-03-19 17:18:03,584.63,141717,601928,5,2854.37,483101
31,0.00,"Малыш и Карлсон, который живет на крыше",977.74,1,2017-03-19 17:18:03,584.63,186906,601928,5,2854.37,483101
32,0.00,Приключения Тома Сойера,585.98,1,2017-03-19 17:18:03,584.63,176818,601928,5,2854.37,483101
33,0.00,Приключения Алисы в Стране Чудес,488.87,1,2017-03-19 17:18:03,584.63,199408,601928,5,2854.37,483101
36,0.00,Макияж. Книга Елены Крыгиной,2212.50,1,2017-03-19 15:49:00,737.50,711378,601903,1,2212.50,911493
42,0.00,Геральт. Последнее желание. Меч Предназначения...,588.75,1,2017-03-19 13:49:00,392.50,189334,601852,2,1594.72,911157
43,0.00,Цири,588.75,1,2017-03-19 13:49:00,392.50,181779,601852,2,1594.72,911157


# 4. Провести построение модели 

# 5. Проанализировать результат
получилось или нет, почему, что можно улучшить, можно ли интерпретировать результат

# 6. Повторить при необходимости 

# Подсказки

* Проводить EDA и работу с признаками на всем датасете
* Для построения модели отсеивать данные тем или иным способом (обосновать выбор) - уменьшить объем данных для анализа
* Оформлять для сдачи задания все эксперименты (в том числе неудачные). Так мы сможем увидеть процесс работы над заданием
* Не тратить 90% времени на первый этап. Планировать время так, чтобы успеть поработать над моделью (в равной степени над каждым пунктом).